In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import copy

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

#### Chuẩn bị dữ liệu

In [2]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = pd.read_excel("period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = pd.read_excel("period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)
group_map_dict = group_map.set_index('code')['group'].to_dict()

#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = [key for key, value in group_map_dict.items() if value == 'tt']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['A', 'B', 'C', 'D']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

In [3]:
#Tạo dict map thời gian và số lượng cổ phiếu
period_map = pd.read_excel("period_data/period_stock_list.xlsx", sheet_name='period_map')
period_map_dict = period_map.set_index('index').apply(lambda row: row.tolist(), axis=1).to_dict()

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
        previous_year = year - 1
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q1"
        previous_year = year
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
        previous_year = year
    else:
        quarter = "q4"
        previous_quarter = "q3"
        previous_year = year
    
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        return f'{previous_quarter}_{previous_year}'

#Xoá đi quý cuối cùng để chỉ tính toán cho các quý lịch sử
pop = period_map_dict.pop(get_quarter('current_quarter'), None)

#Toàn bộ khoảng thời gian tính toán:
calculate_time_span = [period_map_dict['q2_2020'][0], period_map_dict[get_quarter('previous_quarter')][1]]

In [4]:
#Đọc danh sách các cổ phiếu mới được thêm vào quý này
new_stock_list = pd.read_excel("period_data/period_stock_list.xlsx", sheet_name='new_stock_df')['new_stock_list'].tolist()

#Đọc toàn bộ các file csv được xuất ra từ ami eod
full_item_dict = {}
folder_path = 'D:\\t2m-project\\ami-data\\ami_eod_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        full_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

for item, df in full_item_dict.items():
    df['date'] = pd.to_datetime(df['date'], format='%y%m%d')
    full_item_dict[item] = df
                
full_stock_dict = {k:v.drop(['option'], axis=1) for k,v in full_item_dict.items() if (len(k) == 3) & (k not in new_stock_list)}

In [5]:
#Tính toán các đường trung bình và các đường MA
full_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in full_stock_dict.items()}

full_stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma20=df['close'].rolling(window=20, min_periods=1).mean(),
        ma60=df['close'].rolling(window=60, min_periods=1).mean(),
        ma120=df['close'].rolling(window=120, min_periods=1).mean(),
        ma240=df['close'].rolling(window=240, min_periods=1).mean(),
        ma480=df['close'].rolling(window=480, min_periods=1).mean(),
    )
    for key, df in full_stock_dict.items()
}

full_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int),

        liquid_ratio=np.where(df['ma5_V'] != 0, df['volume'] / df['ma5_V'], 0)
    )
    for key, df in full_stock_dict.items()
}

full_stock_dict = {k: v[(v['date'] >= calculate_time_span[0]) & (v['date'] <= calculate_time_span[1])].sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in full_stock_dict.items()}

#### Tạo dict chứa dữ liệu của từng quý

In [6]:
#Thêm cột tên period và số lượng cổ phiếu từng thời kì
def assign_period(x):
    for key, value in period_map_dict.items():
        if (x >= pd.Timestamp(value[0])) & (x <= pd.Timestamp(value[1])):
            return key

for stock, df in full_stock_dict.items():
    df['period'] = df['date'].apply(assign_period)
    df['count'] = df['period'].apply(lambda x: period_map_dict[x][2])

In [7]:
#Bảng danh sách tất cả các cổ phiếu trong tất cả giai đoạn
period_stock_df = pd.read_excel("period_data/stock_classification.xlsx", sheet_name='period_stock_list')

#Xoá đi cột của quý hiện tại để chỉ tính toán cho dữ liệu quá khứ
period_stock_df = period_stock_df.drop(columns=[get_quarter('current_quarter')])

#Dict chứa dữ liệu cổ phiếu cho từng giai đoán
period_stock_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_stock_dict[period] = {k: v[(v['date'] >= period_map_dict[period][0]) & 
                                      (v['date'] <= period_map_dict[period][1])].reset_index(drop=True)
                                      for k, v in copy.deepcopy(full_stock_dict).items()
                                      if k in period_stock_df[period].dropna().tolist()}
    
#Tạo một date_series bao gồm khoảng ngày tính toán eod
full_date_series = pd.DataFrame(full_stock_dict['REE']['date']).rename(columns={0:'date'})    

#Dict chưa date series cho từng giai đoạn
period_date_series_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_date_series_dict[period] = full_date_series[(full_date_series['date'] >= period_map_dict[period][0]) & 
                                                       (full_date_series['date'] <= period_map_dict[period][1])].reset_index(drop=True)

#### Chia nhóm cổ phiếu từng quý

In [8]:
full_stock_classification_df = pd.read_excel("period_data/stock_classification.xlsx", sheet_name='stock_classification')
full_stock_classification_df = full_stock_classification_df[full_stock_classification_df['stock'].isin(full_stock_dict.keys())]

period_stock_classification_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_stock_classification_dict[period] = full_stock_classification_df[full_stock_classification_df['stock'].isin(period_stock_df[period].dropna().tolist())].reset_index(drop=True)
    period_stock_classification_dict[period] = period_stock_classification_dict[period][period_stock_classification_dict[period]['stock'].isin(full_stock_dict.keys())]

    price_arr = []
    cap_arr = []
    for stock in period_stock_classification_dict[period]['stock']:
        df = period_stock_dict[period][stock]
        if len(df) > 0:
            price_arr.append(df['close'].iloc[-1].item())
            cap_arr.append(df['cap'].iloc[-1].item())
        else:
            price_arr.append(0)
            cap_arr.append(0)

    vonhoa_classification_df = period_stock_classification_dict[period].copy()
    vonhoa_classification_df['price'] = price_arr
    vonhoa_classification_df['cap'] = cap_arr

    cap_coef = sum(cap_arr)/10000
    vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
        'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
                ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
                else
        ('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
                ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
                else
        ('large' if x['cap']>=100*cap_coef
                else 'penny'
    )), axis=1)

    period_stock_classification_dict[period] = pd.concat([period_stock_classification_dict[period], vonhoa_classification_df['marketcap_group']], axis=1)


In [9]:
period_all_stock = {}
period_industry_name = {}
period_industry_perform = {}
period_marketcap_group = {}

# Function to create mappings based on category
def create_mapping(stock_dict, category_dict):
    category_map = {}
    for category, stocks in category_dict.items():
        category_map[category] = {stock: stock_dict[stock] for stock in stocks if stock in stock_dict}
    return category_map

for period in period_stock_df.columns[1:].tolist():

    period_all_stock[period] = {}
    period_industry_name[period] = {}
    period_industry_perform[period] = {}
    period_marketcap_group[period] = {}

    stock_by_industry = period_stock_classification_dict[period].set_index('stock')['industry_name'].to_dict()
    stock_by_perform = period_stock_classification_dict[period].set_index('stock')['industry_perform'].to_dict()
    stock_by_marketcap = period_stock_classification_dict[period].set_index('stock')['marketcap_group'].to_dict()

    unique_industries = np.unique(list(stock_by_industry.values()))
    unique_performs = np.unique(list(stock_by_perform.values()))
    unique_marketcaps = ['large', 'mid', 'small', 'penny']

    # Mapping for all_stock
    period_all_stock[period]['all_stock'] = {key: value for key, value in period_stock_dict[period].items()}

    # Mapping for industry
    for industry in unique_industries:
        relevant_stocks = [stock for stock, ind in stock_by_industry.items() if ind == industry]
        period_industry_name[period][industry] = {stock: period_stock_dict[period][stock] for stock in relevant_stocks if stock in period_stock_dict[period]}

    # Mapping for performance
    for performance in unique_performs:
        relevant_stocks = [stock for stock, perf in stock_by_perform.items() if perf == performance]
        period_industry_perform[period][performance] = {stock: period_stock_dict[period][stock] for stock in relevant_stocks if stock in period_stock_dict[period]}

    # Mapping for marketcap
    for marketcap in unique_marketcaps:
        relevant_stocks = [stock for stock, mcap in stock_by_marketcap.items() if mcap == marketcap]
        period_marketcap_group[period][marketcap] = {stock: period_stock_dict[period][stock] for stock in relevant_stocks if stock in period_stock_dict[period]}


#### MS chart cho từng quý và ghép lại

In [10]:
def transform_ms(stock_group, period):
    stock_dict = copy.deepcopy(stock_group)
    # Prepare a base date DataFrame from date_series
    dates_df = period_date_series_dict[period]
    
    for group_name, stocks in stock_dict.items():
        # Initialize a DataFrame for group trends
        group_trends = dates_df.copy()

        # Compute trends across stocks
        for trend in ['trend_5p', 'trend_20p', 'trend_60p', 'trend_120p', 'trend_240p', 'trend_480p']:
            # Concatenate all trend data for current trend across all stocks
            trend_data = pd.concat([stocks[stock][trend] for stock in stocks], axis=1)
            trend_data.fillna(0, inplace=True)
            
            # Calculate the sum and percent for the trend
            sum_trend = trend_data.sum(axis=1)
            percent_trend = sum_trend / len(stocks)
            
            # Add to group trends DataFrame
            group_trends[f'{trend}'] = percent_trend

        stock_dict[group_name] = group_trends.sort_values('date', ascending=False)
        
    return stock_dict

In [11]:
#Tạo các bảng MS cho từng giai đoạn
period_all_stock_ms = {}
period_industry_name_ms = {}
period_industry_perform_ms = {}
period_marketcap_group_ms = {}

for period in period_stock_df.columns[1:].tolist():
    period_all_stock_ms[period] = transform_ms(period_all_stock[period], period)
    period_industry_name_ms[period] = transform_ms(period_industry_name[period], period)
    period_industry_perform_ms[period] = transform_ms(period_industry_perform[period], period)
    period_marketcap_group_ms[period] = transform_ms(period_marketcap_group[period], period)

In [12]:
#Ghép các bảng MS vào chung 1 bảng full lịch sử
all_stock_ms = {}
industry_name_ms = {}
industry_perform_ms = {}
marketcap_group_ms = {}

for group in all_stock_key_list:
    all_stock_ms[group] = pd.DataFrame()
    for period in period_stock_df.columns[1:].tolist():
        all_stock_ms[group] = pd.concat([all_stock_ms[group], period_all_stock_ms[period][group]])

for group in industry_name_list:
    industry_name_ms[group] = pd.DataFrame()
    for period in period_stock_df.columns[1:].tolist():
        industry_name_ms[group] = pd.concat([industry_name_ms[group], period_industry_name_ms[period][group]])
    industry_name_ms[group] = industry_name_ms[group]

for group in industry_perform_list:
    industry_perform_ms[group] = pd.DataFrame()
    for period in period_stock_df.columns[1:].tolist():
        industry_perform_ms[group] = pd.concat([industry_perform_ms[group], period_industry_perform_ms[period][group]])
    industry_perform_ms[group] = industry_perform_ms[group]

for group in marketcap_group_list:
    marketcap_group_ms[group] = pd.DataFrame()
    for period in period_stock_df.columns[1:].tolist():
        marketcap_group_ms[group] = pd.concat([marketcap_group_ms[group], period_marketcap_group_ms[period][group]])
    marketcap_group_ms[group] = marketcap_group_ms[group]

#Gộp tất cả biểu đồ MS vào 1 bảng
full_group_ms_chart_df = pd.DataFrame()
for item in [all_stock_ms, industry_name_ms, industry_perform_ms, marketcap_group_ms]:
    for group, df in item.items():
        df['name'] = group
        full_group_ms_chart_df = pd.concat([full_group_ms_chart_df, df], axis=0)

#### Điểm dòng tiền từng cổ phiếu full lịch sử

In [13]:
def score_calculation(row):
    try:
        return (((row['close'] - row['low']) - (row['high'] - row['close'])) / (row['high'] - row['low']) *
                abs((row['close'] - row['close_prev'])) / row['close_prev'] *
                (row['volume']*row['close']) / (row['ma5_prev'] * row['ma5_V'])) * 100 \
                + ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng
    except ZeroDivisionError:
        return ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng


In [14]:
#Tính cho toàn bộ lịch sử
raw_stock_score_dict = {}
for stock in full_stock_dict.keys():
    #Lọc ra các cột cần sử dụng
    temp_df = full_stock_dict[stock][['stock', 'date', 'period', 'count', 'high', 'low', 'close', 'volume', 'liquid_ratio', 'ma5', 'ma5_V']]
    #Tính điểm dòng tiền t0 và t5
    temp_df['ma5_prev'] = temp_df['ma5'].shift(-1)
    temp_df['close_prev'] = temp_df['close'].shift(-1)
    temp_df['t0_score'] = temp_df.apply(score_calculation, axis=1)
    temp_df['t5_score'] = temp_df['t0_score'][::-1].rolling(window=5, min_periods=1).mean()[::-1]

    temp_df = temp_df.drop(columns=['ma5_prev','close_prev'])
    #Gán lại temp_df cho dict
    raw_stock_score_dict[stock] = temp_df

In [15]:
#Tách điểm dòng tiền vào từng giai đoạn để tính xếp hạng
period_stock_score_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_stock_score_dict[period] = {k: v[(v['date'] >= period_map_dict[period][0]) & 
                                            (v['date'] <= period_map_dict[period][1])].reset_index(drop=True)
                                            for k, v in copy.deepcopy(raw_stock_score_dict).items()
                                            if k in period_stock_df[period].dropna().tolist()}

#Tính xếp hạng cổ phiếu cho từng giai đoạn
for period in period_stock_df.columns[1:].tolist():
    the_dict = period_stock_score_dict[period]

    #Tính xếp hạng cho cổ phiếu
    t0_ranking_df = period_date_series_dict[period].copy()
    t5_ranking_df = period_date_series_dict[period].copy()
    for stock in the_dict.keys():
        t0_ranking_df[stock] = the_dict[stock]['t0_score']
        t0_ranking_df.fillna(0, inplace=True)
        t5_ranking_df[stock] = the_dict[stock]['t5_score']
        t5_ranking_df.fillna(0, inplace=True)
    t0_ranking_df = t0_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)
    t5_ranking_df = t5_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)

    #Ghép xếp hạng vào bảng thông tin cổ phiếu
    for stock, df in the_dict.items():
        df['rank_t0'] = t0_ranking_df[stock]
        df['rank_t5'] = t5_ranking_df[stock]
        
        #Check xem xếp hạng T0 nằm trong top 10% hay không
        df['top_check'] = df.apply(lambda x: 1 if x['rank_t0'] <= x['count']*0.1 else 0, axis=1)

In [16]:
#Ghép lại bảng điểm dòng tiền cho tất cả giai đoạn
concat_stock_score_dict = {}
for stock in full_stock_classification_df['stock'].tolist():
    temp_df = pd.DataFrame()
    for period in period_stock_df.columns[1:].tolist():
        if stock in period_stock_df[period].tolist():
            if stock == "HMR":
                print(period)
            temp_df = pd.concat([temp_df, period_stock_score_dict[period][stock]], axis=0)

    #Gán lại temp_df cho dict dòng tiền tổng
    concat_stock_score_dict[stock] = temp_df.reset_index(drop=True)

q3_2024
q4_2024


In [17]:
#Merge bảng và xử lý các giai đoạn không có cổ phiếu
full_stock_score_dict = {}
for stock, df in raw_stock_score_dict.items():

    #Nếu trong lịch sử có giai đoạn lọt top thì ghép thêm vào, nếu ko thì thôi
    if (stock in concat_stock_score_dict) and (len(concat_stock_score_dict[stock]) > 0):
        df = df.merge(concat_stock_score_dict[stock][['date','rank_t0','rank_t5','top_check']], on='date', how='left')
    
        df['rank_t0'] = df.apply(lambda x: x['rank_t0'] if pd.notnull(x['rank_t0']) else x['count'], axis=1)
        df['rank_t5'] = df.apply(lambda x: x['rank_t5'] if pd.notnull(x['rank_t5']) else x['count'], axis=1)
        df['top_check'] = df['top_check'].fillna(0)

        #Đếm số phiên lọt top 10%
        df['top_count'] = df['top_check'][::-1].rolling(window=20, min_periods=1).sum()[::-1]

        full_stock_score_dict[stock] = df

#### Điểm dòng tiền nhóm cổ phiếu

In [18]:
#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
def adjust_score_by_breath(t0_score, ratio_column):
    adjusted_score = []
    for score, ratio in zip(t0_score, ratio_column):
        if score >= 0:
            adjusted_score.append(score*ratio)
        else:
            adjusted_score.append(score*(1-ratio))
    return adjusted_score

#Hàm điều chỉnh điểm dòng tiền của cổ phiếu tránh sự đột biến khi đóng góp vào nhóm chung
def adjust_score_for_smooth(row, column_name, max_percent, mark):
    origin_score = row[column_name]
    
    if abs(origin_score) > row['total'] * max_percent:

        sum_abs = row['total'] - abs(row[column_name])
        fixed_score = sum_abs / (1 - max_percent) - sum_abs

        if origin_score >= 0:
            return fixed_score
        else:
            return -fixed_score
    else:
        mark[0] = 0
        return origin_score

#Áp dụng hàm điều chỉnh điểm phía trên vào các nhóm cổ phiếu, việc này lặp lại nhiều lần cho tới khi triệt tiêu sự đột biến
def apply_smooth_score(period_score_dict, group_type, period):

    #Lấy ra dict điểm dòng tiền và danh sách cổ phiếu của period
    score_dict = period_score_dict[period]
    period_stock_classification_df = period_stock_classification_dict[period].copy()

    #Lấy ra danh sách cách nhóm nhỏ của mõi cách chia cổ phiếu
    if group_type == 'all_stock':
        key_list = all_stock_key_list
    elif group_type == 'industry_perform':
        key_list = [key for key, value in group_map_dict.items() if value == 'hs']
    elif group_type == 'marketcap_group':
        key_list = [key for key, value in group_map_dict.items() if value == 'cap'] 
    elif group_type == 'industry_name':
        key_list = [key for key, value in group_map_dict.items() if value in ['A', 'B', 'C', 'D']]

    for key in key_list:
        score_df = period_date_series_dict[period].copy()
        if group_type == 'all_stock':
            stock_list = period_stock_classification_df['stock'].tolist()
        else:
            stock_list = [stock for stock in 
            period_stock_classification_df[period_stock_classification_df[group_type]==key]['stock'].dropna().tolist()]
        for stock in stock_list:
            try: score_df[stock] = score_dict[stock][f't0_score']
            except: pass

        max_percent = max(0.1, min(5*(1/len(stock_list)), 0.5))
        score_df['total'] = score_df.iloc[:, 1:].abs().sum(axis=1)

        mark = [1]
        while True:
            if mark[0] == 1:
                for stock in stock_list:
                    score_df[stock] = score_df.iloc[:, 1:].apply(adjust_score_for_smooth, axis=1, args=(stock, max_percent, mark))
            if mark[0] == 0: break

        for stock in stock_list:
            try: score_dict[stock][f't0_{group_type}'] = score_df[stock]
            except: pass

In [19]:
#Thêm các cột dòng tiền đóng góp vào các nhóm cổ phiếu vào các dict period (đã loại bỏ đột biến)
for period in period_stock_df.columns[1:].tolist():
    for group_type in ['all_stock','industry_name','industry_perform','marketcap_group']:
        apply_smooth_score(period_stock_score_dict, group_type, period)

In [20]:
#Tính độ rộng dòng tiền các nhóm cổ phiếu từng phiên trong từng giai đoạn
period_market_breath_dict = {}
for period in period_stock_df.columns[1:].tolist():

    period_market_breath_dict[period] = {}
    period_stock_classification_df = period_stock_classification_dict[period].copy()
    full_stock_score_df = period_date_series_dict[period].copy()

    for stock, df in period_stock_score_dict[period].items():
        full_stock_score_df[stock] = period_stock_score_dict[period][stock]['t0_score']
    full_stock_score_df.iloc[:,1:] = full_stock_score_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

    all_stock_breadth_dict = {}
    for key in all_stock_key_list:
        stock_list = period_stock_classification_df['stock'].tolist()
        all_stock_breadth_dict[key] = full_stock_score_df[['date'] + [columns for columns in stock_list]]
        period_market_breath_dict[period][key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)
    
    industry_name_breadth_dict = {}
    for key in industry_name_list:
        stock_list = period_stock_classification_df['stock'].tolist()
        industry_name_breadth_dict[key] = full_stock_score_df[['date'] + [columns for columns in stock_list]]
        period_market_breath_dict[period][key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

    industry_perform_breadth_dict = {}
    for key in industry_perform_list:
        stock_list = period_stock_classification_df['stock'].tolist()
        industry_perform_breadth_dict[key] = full_stock_score_df[['date'] + [columns for columns in stock_list]]
        period_market_breath_dict[period][key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

    marketcap_group_breadth_dict = {}
    for key in marketcap_group_list:
        stock_list = period_stock_classification_df['stock'].tolist()
        marketcap_group_breadth_dict[key] = full_stock_score_df[['date'] + [columns for columns in stock_list]]
        period_market_breath_dict[period][key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)
    
#Chỉnh sửa lại điểm dòng tiền trực tiếp vào trong period_stock_score_dict
for period in period_stock_df.columns[1:].tolist():
    period_stock_classification_df = period_stock_classification_dict[period].copy()
    for stock, df in period_stock_score_dict[period].items():
        for group_type in ['all_stock','industry_name','industry_perform','marketcap_group']:
            if group_type == 'all_stock':
                df[f't0_{group_type}'] = adjust_score_by_breath(df[f't0_{group_type}'], period_market_breath_dict[period]['all_stock'])
            else:
                group_name = period_stock_classification_df[period_stock_classification_df['stock']==stock][group_type].item()
                df[f't0_{group_type}'] = adjust_score_by_breath(df[f't0_{group_type}'], period_market_breath_dict[period][group_name])

In [21]:
#Tạo dict dòng tiền các nhóm cổ phiếu từng giai đoạn
period_group_score_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_group_score_dict[period] = period_date_series_dict[period].copy()
    period_full_stock_df = period_stock_classification_dict[period].copy()
    period_stock_classification_df = period_stock_classification_dict[period].copy()

    #Thêm cột điểm dòng tiền toàn bộ cổ phiếu
    for key in all_stock_key_list:
        score_df = period_date_series_dict[period].copy()
        for stock in period_stock_classification_df['stock']:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_all_stock']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        period_group_score_dict[period][key] = score_df['total']

    #Thêm các cột điểm dòng tiền ngành
    for nganh in industry_name_list:
        score_df = period_date_series_dict[period].copy()
        for stock in period_stock_classification_df[period_stock_classification_df['industry_name']==nganh]['stock']:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_industry_name']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        period_group_score_dict[period][nganh] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm hiệu suất
    for group in industry_perform_list:
        score_df = period_date_series_dict[period].copy()
        for stock in period_stock_classification_df[period_stock_classification_df['industry_perform']==group]['stock']:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_industry_perform']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        period_group_score_dict[period][group] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm vốn hoá
    for marketcap in marketcap_group_list:
        score_df = period_date_series_dict[period].copy()
        for stock in period_stock_classification_df[period_stock_classification_df['marketcap_group']==marketcap]['stock']:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_marketcap_group']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        period_group_score_dict[period][marketcap] = score_df['total']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full
full_group_score_df = pd.DataFrame()
for period in period_stock_df.columns[1:].tolist():
    full_group_score_df = pd.concat([full_group_score_df, period_group_score_dict[period]]).sort_values('date', ascending=False).reset_index(drop=True)

full_group_score_df = full_group_score_df.fillna(0)

In [22]:
def mean_of_positive_values(df, length):
    positive_values = df[df >= 0]  # Lọc ra các giá trị âm
    return positive_values.sum(axis=1)/length

#Tạo dict dòng tiền các nhóm cổ phiếu từng giai đoạn
period_group_score_positive_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_group_score_positive_dict[period] = period_date_series_dict[period].copy()
    period_full_stock_df = period_stock_classification_dict[period].copy()
    period_stock_classification_df = period_stock_classification_dict[period].copy()

    #Thêm cột điểm dòng tiền toàn bộ cổ phiếu
    for key in all_stock_key_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_all_stock']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_positive_dict[period][key] = score_df['total']

    #Thêm các cột điểm dòng tiền ngành
    for nganh in industry_name_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df[period_stock_classification_df['industry_name']==nganh]['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_industry_name']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_positive_dict[period][nganh] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm hiệu suất
    for group in industry_perform_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df[period_stock_classification_df['industry_perform']==group]['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_industry_perform']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_positive_dict[period][group] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm vốn hoá
    for marketcap in marketcap_group_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df[period_stock_classification_df['marketcap_group']==marketcap]['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_marketcap_group']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_positive_dict[period][marketcap] = score_df['total']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full
full_group_score_positive_df = pd.DataFrame()
for period in period_stock_df.columns[1:].tolist():
    full_group_score_positive_df = pd.concat([full_group_score_positive_df, period_group_score_positive_dict[period]]).sort_values('date', ascending=False).reset_index(drop=True)

full_group_score_positive_df = full_group_score_positive_df.fillna(0)

In [23]:
def mean_of_negative_values(df, length):
    negative_values = df[df < 0]  # Lọc ra các giá trị âm
    return negative_values.sum(axis=1)/length

#Tạo dict dòng tiền các nhóm cổ phiếu từng giai đoạn
period_group_score_negative_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_group_score_negative_dict[period] = period_date_series_dict[period].copy()
    period_full_stock_df = period_stock_classification_dict[period].copy()
    period_stock_classification_df = period_stock_classification_dict[period].copy()

    #Thêm cột điểm dòng tiền toàn bộ cổ phiếu
    for key in all_stock_key_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_all_stock']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_negative_dict[period][key] = score_df['total']

    #Thêm các cột điểm dòng tiền ngành
    for nganh in industry_name_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df[period_stock_classification_df['industry_name']==nganh]['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_industry_name']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_negative_dict[period][nganh] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm hiệu suất
    for group in industry_perform_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df[period_stock_classification_df['industry_perform']==group]['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_industry_perform']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_negative_dict[period][group] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm vốn hoá
    for marketcap in marketcap_group_list:
        score_df = period_date_series_dict[period].copy()
        temp_stock_list = period_stock_classification_df[period_stock_classification_df['marketcap_group']==marketcap]['stock']
        for stock in temp_stock_list:
            score_df[stock] = period_stock_score_dict[period][stock]['t0_marketcap_group']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        period_group_score_negative_dict[period][marketcap] = score_df['total']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full
full_group_score_negative_df = pd.DataFrame()
for period in period_stock_df.columns[1:].tolist():
    full_group_score_negative_df = pd.concat([full_group_score_negative_df, period_group_score_negative_dict[period]]).sort_values('date', ascending=False).reset_index(drop=True)

full_group_score_negative_df = full_group_score_negative_df.fillna(0)

#### Tổng thanh khoản nhóm cổ phiếu

In [24]:
#Tạo dict chứa thanh khoản trung bình của mỗi nhóm cổ phiếu trong các giai đoạn
period_group_liquidity_dict = {}

for period in period_stock_df.columns[1:].tolist():

    period_group_liquidity_dict[period] = period_date_series_dict[period].copy()
    period_date_series = period_date_series_dict[period].copy()
    period_full_stock_df = period_stock_classification_dict[period].copy()
    period_stock_classification_df = period_stock_classification_dict[period].copy()
    
    for name in all_stock_key_list:
        temp_volume_df = period_date_series.copy()
        for stock, df in period_all_stock[period][name].items():
            temp_volume_df[stock] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        period_group_liquidity_dict[period][name] = temp_volume_df['volume']

    for name in industry_name_list:
        temp_volume_df = period_date_series.copy()
        for stock, df in period_industry_name[period][name].items():
            temp_volume_df[stock] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        period_group_liquidity_dict[period][name] = temp_volume_df['volume']

    for name in industry_perform_list:
        temp_volume_df = period_date_series.copy()
        for stock, df in period_industry_perform[period][name].items():
            temp_volume_df[stock] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        period_group_liquidity_dict[period][name] = temp_volume_df['volume']

    for name in marketcap_group_list:
        temp_volume_df = period_date_series.copy()
        for stock, df in period_marketcap_group[period][name].items():
            temp_volume_df[stock] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        period_group_liquidity_dict[period][name] = temp_volume_df['volume']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full thanh khoản
full_group_liquidity_df = pd.DataFrame()
for period in period_stock_df.columns[1:].tolist():
    full_group_liquidity_df = pd.concat([full_group_liquidity_df, period_group_liquidity_dict[period]]).sort_values('date', ascending=False).reset_index(drop=True)

#### Price Index các nhóm cổ phiếu

In [25]:
def calculate_total_change(stock_group, name, price_index_date_series):
    period_index_df = price_index_date_series.copy()

    for stock, df in stock_group[name].items():
        period_index_df[stock] = df['close']
        period_index_df[stock] = period_index_df[stock][::-1].pct_change()[::-1]

    period_index_df['total_change'] = period_index_df.iloc[:,1:].sum(axis=1)
    period_index_df['total_change'] = ((period_index_df['total_change']/len(stock_group[name]))*100)
    period_index_df['total_change'] = period_index_df['total_change']*10

    return period_index_df['total_change']

In [26]:
period_group_price_index_dict = {}
for period in period_stock_df.columns[1:].tolist():

    period_date_series = period_date_series_dict[period].copy()
    temp_df = period_date_series.copy()

    for key in all_stock_key_list:
        temp_df[key] = calculate_total_change(period_all_stock[period], key, period_date_series)

    for key in industry_name_list:
        temp_df[key] = calculate_total_change(period_industry_name[period], key, period_date_series)

    for key in industry_perform_list:
        temp_df[key] = calculate_total_change(period_industry_perform[period], key, period_date_series)

    for key in marketcap_group_list:
        temp_df[key] = calculate_total_change(period_marketcap_group[period], key, period_date_series)

    period_group_price_index_dict[period] = temp_df

full_group_price_change_df= pd.DataFrame()
for period in period_stock_df.columns[1:].tolist():
    full_group_price_change_df = pd.concat([full_group_price_change_df, period_group_price_index_dict[period]]).sort_values('date', ascending=False).reset_index(drop=True)

#### Lưu lại dữ liệu

In [27]:
with pd.ExcelWriter('period_data/period_processed_data.xlsx', engine='openpyxl') as writer:
    full_group_ms_chart_df.to_excel(writer, sheet_name='full_group_ms_chart_df', index=False)
    full_group_liquidity_df.to_excel(writer, sheet_name='full_group_liquidity_df', index=False)
    full_group_price_change_df.to_excel(writer, sheet_name='full_group_price_change_df', index=False)
    full_group_score_df.to_excel(writer, sheet_name='full_group_score_df', index=False)
    full_group_score_positive_df.to_excel(writer, sheet_name='full_group_score_positive_df', index=False)
    full_group_score_negative_df.to_excel(writer, sheet_name='full_group_score_negative_df', index=False)